# 概要

Character-LevelのCNNでWAFを作る。  
論文の内容をKerasで実装してみる。  
原著論文はこちら http://iyatomi-lab.info/sites/default/files/user/CSPA2018%20Proceedings_ito.pdf 


## データの準備

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Embedding
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Add, Reshape, normalization
from keras import backend as K

In [50]:
%cd /content/drive/MyDrive/WAffle/
!ls -al

/content/drive/MyDrive/WAffle
total 24
-rw------- 1 root root   151 Dec 18 14:37 2020-12-18.gslides
-rw------- 1 root root 14446 Dec 29 09:15 CharacterLevel-CNN
drwx------ 4 root root  4096 Dec 27 18:45 Dataset
drwx------ 2 root root  4096 Dec 27 13:42 KerasPractice
-rw------- 1 root root   151 Oct 26 12:09 WAF_Overview_補足資料.gslides


In [51]:
df = pd.read_csv('Dataset/cisc_database/csic_database.csv')

In [52]:
df = df.rename(columns={'Unnamed: 0':'Target'})

In [53]:
# Targetカラムの変更。
# if (df['Target'] == Normal){0} else {1}

df['Target'] = df['Target'].apply(lambda x:0 if str(x) == 'Normal' else 1)
df.head()

,Target,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,classification,URL
0,0,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/index.jsp HTTP/1.1
1,0,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=81761ACA043B0E6014CA42A4BCD06AB5,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/anadir.j...
2,0,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=933185092E0B668B90676E0A2B0767AF,application/x-www-form-urlencoded,Connection: close,Content-Length: 68,id=3&nombre=Vino+Rioja&precio=100&cantidad=55&...,0,http://localhost:8080/tienda1/publico/anadir.j...
3,0,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=8FA18BA82C5336D03D3A8AFA3E68CBB0,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/autentic...
4,0,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=7104E6C68A6BCF1423DAE990CE49FEE2,application/x-www-form-urlencoded,Connection: close,Content-Length: 63,modo=entrar&login=choong&pwd=d1se3ci%F3n&remem...,0,http://localhost:8080/tienda1/publico/autentic...


In [54]:
X, y = train_test_split(df, train_size=0.8, test_size=0.2, random_state=42)

In [55]:
X_train, X_test = train_test_split(X, train_size=0.8, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y, train_size=0.8, test_size=0.2, random_state=42)

In [56]:
print('x_train shape', x_train.shape)
print('y_train shape', y_train.shape)
print('x_test shape', x_test.shape)
print('y_test shape', y_test.shape)

x_train shape (39081, 17)
y_train shape (9770, 17)
x_test shape (9771, 17)
y_test shape (2443, 17)


## アーキテクチャ

1. Input(1000 characters)
2. Embedding(128demensions)
3. Conv(kernelsize = K, filter_num = 64) -> RELU
4. Max Pooling(kernelsize = K)
5. Conv(kernelsize = K, filter_num = 64) -> RELU
6. Max Pooling(kernel_size = size of Conv 5's output)
7. concat output & reshape into 64 length vector
8. FCN(64 units) -> RELU
9. Batch normalize
10. Dropout(0.5)
11. FCN(1 unit) -> Sigmoid

(categorical_crossentropy)

In [ ]:
#いつか使う URL decode
import urllib.parse
decoded_URL = urllib.parse.unquote(encoded_URL)

In [ ]:
# 文字列の特徴ベクトル変換
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

In [62]:
# config
input_max_size = 1000
embeddinf_size = 128
conv_layers = 4
kernel_sizes = [4,5,6,7]
filter_num = 64
fully_connected_layers = [64,64]
num_of_classes = 2
dropout = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [66]:
def create_model(embed_size, input_max_size, filter_sizes, filter_num):
  # Input Layer
  # URLdecodeされた文字列をInputとして与える。
  inputs = Input(shape=(input_size,), name='character_input', dtype='int')

  # Embedding Layer
  x = Embedding(alphabet_size +1, embedding_size, input_length=input_size)(inputs)
  x = Reshape((input_max_size, embeddinf_size, 1))(x)

  # Convolution Layers
  convolution_output = []
  for conv_layer in conv_layers:
    y = Conv1D(kernel_size=kernel_sizes[conv_layer], filters=filter_num)(x)
    y = MaxPooling1D(pool_size=kernel_sizes[conv_layer])(y)
    y = Conv1D(kernel_size=kernel_sizes[conv_layer], filters=filter_num, activation='relu')(y)
    y = MaxPooling1D(pool_size=x)(y)
    convolution_output.append(y)

  # concat output
  x = Add()(convolution_output)

  # reshape into 64 length vector
  x = Reshape(256)(x)

  # Fully Connected Layers
  # x = Flatten()(x)
  x = Dense(64, activation='relu')(x)

  # Batch Normalization
  x = normalization.BatchNormalization()(x)

  # DropOut
  x = Dropout(dropout)(x)

  # Fully Connected Layers
  predictions = Dense(1, activation='softmax')

  model = Model(input=inputs, outputs=predictions)
  return model